In [ ]:
import os
%pwd


In [ ]:
os.chdir('../')
%pwd

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataProcessingConfig:
    training_org_img_folder: Path
    training_segment_img_folder: Path
    testing_org_img_folder: Path
    testing_segment_img_folder: Path
    number_of_classes: int



@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    base_model: Path
    number_of_classes: int
    image_size: list
    batch_size: int
    epochs: int
    learning_rate: float
    backbone: str

In [4]:


from src.imageSegmentation.constants import *
from src.imageSegmentation.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_processing_config(self) -> DataProcessingConfig:
        config = self.config.data_preprocessing
        
        data_processing_config = DataProcessingConfig(
            training_org_img_folder = Path(config.training_org_img_folder),
            training_segment_img_folder = Path(config.training_segment_img_folder),
            testing_org_img_folder = Path(config.testing_org_img_folder),
            testing_segment_img_folder = Path(config.testing_segment_img_folder),
            number_of_classes = self.params.CLASSES
        )
        return data_processing_config
    


    def get_training_config(self) -> ModelTrainingConfig:
        
        config = self.config.model_training

        create_directories([config.root_dir])

        training_config = ModelTrainingConfig(
            root_dir = Path(config.root_dir),
            base_model= Path(config.base_model),
            number_of_classes = self.params.CLASSES,
            image_size = self.params.IMAGE_SIZE,
            batch_size = self.params.BATCH_SIZE,
            epochs = self.params.EPOCHS,
            learning_rate = self.params.LEARNING_RATE,
            backbone = self.params.BACKBONE
        )
        return training_config

In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import cv2
import numpy as np
from tensorflow import keras
import segmentation_models as sm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import load_img, img_to_array
from keras.layers import Input, Conv2D, Dropout, MaxPooling2D, BatchNormalization, UpSampling2D, Concatenate, concatenate, Conv2DTranspose
from keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import to_categorical

class DataPreprocess:
    def __init__(self, config: DataProcessingConfig):
        self.config = config


    def convert_org_img_to_array(self):
        org_folder = self.config.training_org_img_folder
        x = []
        for file in os.listdir(org_folder):
            path1 = os.path.join(org_folder, file)
            img1 = cv2.imread(path1, 1)
            img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
            img1 = cv2.resize(img1, (512, 512))
            x.append(img1)
        x = np.array(x)
        return x

    def convert_seg_img_to_array(self):
        seg_folder = self.config.training_segment_img_folder
        y = []
        for file2 in os.listdir(seg_folder):
            path2 = os.path.join(seg_folder, file2)
            img2 = np.array(cv2.imread(path2, 1))
            img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
            img2 = cv2.resize(img2, (512, 512))
            y.append(img2)
        y = np.array(y)
        return y


    @staticmethod
    def rgb_to_2D_label(image):
        image_seg = np.zeros(image.shape, dtype= np.uint8)

        image_seg[np.all(image == np.array([255, 235, 0]), axis= -1)] = 0
        image_seg[np.all(image == np.array([0, 0 ,0]), axis= -1)] = 1
        image_seg[np.all(image == np.array([27 ,71, 151]), axis= -1)] = 2
        image_seg[np.all(image == np.array([111, 48, 253]), axis= -1)] = 3
        image_seg[np.all(image == np.array([137, 126, 126]), axis= -1)] = 4
        image_seg[np.all(image == np.array([201, 19, 223]), axis= -1)] = 5
        image_seg[np.all(image == np.array([254, 233, 3]), axis= -1)] = 6
        image_seg[np.all(image == np.array([255, 0, 29]), axis= -1)] = 7
        image_seg[np.all(image == np.array([255, 159, 0]), axis= -1)] = 8
        image_seg[np.all(image == np.array([255, 160, 1]), axis= -1)] = 9

        image_seg = image_seg[:,:,0]
        return image_seg
    
    
    def rgb_to_class_num(self):
        segArray = self.convert_seg_img_to_array()
        labels_list = []                             
        for i in range(segArray.shape[0]):
            label = self.rgb_to_2D_label(segArray[i])
            labels_list.append(label)
        labels_arr = np.array(labels_list)
        labels_arr = np.expand_dims(labels_arr, axis=3)
        return labels_arr
    
    def convert_to_categorical(self):
        imgArray = self.rgb_to_class_num()
        classes = self.config.number_of_classes
        label_categorical = to_categorical(imgArray, num_classes = classes)
        return label_categorical

[2023-07-27 12:16:26,451: INFO: utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2023-07-27 12:16:26,451: INFO: utils: NumExpr defaulting to 8 threads.]
Segmentation Models: using `keras` framework.


In [6]:
def bce_jaccard_loss(y_train, y_valid, smooth=1e-7):
    y_train_flat = K.flatten(y_train)
    y_valid_flat = K.flatten(y_valid)

    bce_loss = K.binary_crossentropy(y_train_flat, y_valid_flat)

    intersection = K.sum(y_train_flat * y_valid_flat)
    union = K.sum(y_train_flat) + K.sum(y_valid_flat) - intersection
    jaccard_loss = 1.0 - (intersection + smooth) / (union + smooth)
    total_loss = bce_loss + jaccard_loss
    return total_loss

def iou_score(y_train, y_valid, smooth=1e-7):

    y_train_flat = K.flatten(y_train)
    y_valid_flat = K.flatten(y_valid)

    intersection = K.sum(y_train_flat * y_valid_flat)
    union = K.sum(y_train_flat) + K.sum(y_valid_flat) - intersection

    iou = (intersection + smooth) / (union + smooth)
    return iou
    


class ModelTraining:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    def spliting_the_dataset(self, training, testing):
        x_train, x_valid , y_train, y_valid = train_test_split(training, testing, test_size= 0.15, random_state= 0)
        preprocess_input = sm.get_preprocessing(self.config.backbone)
        X_train = preprocess_input(x_train)
        X_valid = preprocess_input(x_valid)
        return X_train, X_valid , y_train, y_valid
    
    def model_build(self):

        input_shape = self.config.image_size 
        num_classes = self.config.number_of_classes
        
        # print(input_shape)
        # print(num_classes)

        inputs = Input(input_shape)
        # Downsampling path
        conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)
        conv1 = Conv2D(32, 3, activation='relu', padding='same')(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = Conv2D(64, 3, activation='relu', padding='same')(pool1)
        conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

        conv3 = Conv2D(128, 3, activation='relu', padding='same')(pool2)
        conv3 = Conv2D(128, 3, activation='relu', padding='same')(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

        conv4 = Conv2D(256, 3, activation='relu', padding='same')(pool3)
        conv4 = Conv2D(256, 3, activation='relu', padding='same')(conv4)
        drop4 = Dropout(0.5)(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

        conv5 = Conv2D(512, 3, activation='relu', padding='same')(pool4)
        conv5 = Conv2D(512, 3, activation='relu', padding='same')(conv5)
        drop5 = Dropout(0.5)(conv5)

        # Upsampling path
        up6 = Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(drop5)
        up6 = concatenate([up6, drop4])
        conv6 = Conv2D(256, 3, activation='relu', padding='same')(up6)
        conv6 = Conv2D(256, 3, activation='relu', padding='same')(conv6)

        up7 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv6)
        up7 = concatenate([up7, conv3])
        conv7 = Conv2D(128, 3, activation='relu', padding='same')(up7)
        conv7 = Conv2D(128, 3, activation='relu', padding='same')(conv7)

        up8 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv7)
        up8 = concatenate([up8, conv2])
        conv8 = Conv2D(64, 3, activation='relu', padding='same')(up8)
        conv8 = Conv2D(64, 3, activation='relu', padding='same')(conv8)

        up9 = Conv2DTranspose(32, 2, strides=(2, 2), padding='same')(conv8)
        up9 = concatenate([up9, conv1])
        conv9 = Conv2D(32, 3, activation='relu', padding='same')(up9)
        conv9 = Conv2D(32, 3, activation='relu', padding='same')(conv9)

        outputs = Conv2D(num_classes, 1, activation='softmax')(conv9)

        model = Model(inputs=inputs, outputs=outputs)

        return model


    def compile_model(self, image_size: int, number_of_classes: int, learning_rate: float):
        model = self.model_build()  
        optimizer = Adam(learning_rate = learning_rate)
        # model.compile(optimizer= optimizer, loss= categorical_crossentropy, metrics= ['accuracy'])
                ## for custom loss and metrics
        model.compile(optimizer=optimizer, loss=bce_jaccard_loss, metrics=[iou_score])
        return model
    
    
    def fit_model(self, X_train, X_valid , y_train, y_valid):
        model = self.compile_model(self.config.image_size, self.config.number_of_classes, self.config.learning_rate)
        model.fit(X_train, y_train, batch_size= self.config.batch_size, epochs= self.config.epochs, validation_data= (X_valid, y_valid))
        self.save_model(self.config.base_model, model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        # model.save(path, model)
        model.save(path)

In [7]:

try:
    config = ConfigurationManager()
    data_process_config = config.get_data_processing_config()
    data_process = DataPreprocess(config = data_process_config)
    org_img_arr = data_process.convert_org_img_to_array()
    seg_img_arr = data_process.convert_seg_img_to_array()
    y_cls = data_process.rgb_to_class_num()
    y_cat = data_process.convert_to_categorical()

    training_config = config.get_training_config()
    training = ModelTraining(config= training_config)
    X_train, X_valid , y_train, y_valid = training.spliting_the_dataset(org_img_arr, y_cat)
    fitting = training.fit_model(X_train, X_valid , y_train, y_valid)
except Exception as e:
    raise e
    
    
    

[2023-07-27 12:16:27,222: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-27 12:16:27,231: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-27 12:16:27,231: INFO: common: created directory at: artifacts]
(12, 512, 512, 3) ..........1
(12, 512, 512, 3) ..........2
(12, 512, 512, 1) ..........3
(12, 512, 512, 10) ..........4
[2023-07-27 12:16:29,116: INFO: common: created directory at: artifacts/segmentation_model]
(10, 512, 512, 3) (2, 512, 512, 3) (10, 512, 512, 10) (2, 512, 512, 10)
[512, 512, 3]
10
Epoch 1/2
2/2 [==============================] - 31s 7s/step - loss: 243613.4688 - iou_score: 0.1788 - val_loss: 2390.8704 - val_iou_score: 0.4193
Epoch 2/2
2/2 [==============================] - 21s 5s/step - loss: 3615.2917 - iou_score: 0.0252 - val_loss: 118.0648 - val_iou_score: 0.1071


In [8]:
## sg_model: tf.keras.model, 